In [1]:
# 데이터프레임에서 Contents 컬럼 Series 가져오기
import pandas as pd
import pandas_datareader as wb
import datetime
import matplotlib.pyplot as plt

# 뉴스 데이터 로드
news_data = pd.read_csv('./사회_20230707_143938_date4011_4001.csv')  # 뉴스 데이터 파일 경로에 맞게 수정
stopwords = pd.read_csv('./stopword.txt')
st = stopwords['text'].tolist()
contents = news_data['content']

import re
 
def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    return text  


In [2]:
import FinanceDataReader as fdr

krx = fdr.StockListing('KRX')
NASDAQ = fdr.StockListing('NASDAQ')

KS11 = fdr.DataReader('^KS11') ## 코스피지수 종합
KS11_2023 = fdr.DataReader('^KS11','2023')
KS11_2021_2023 = fdr.DataReader('^KS11','2021-06-28','2023-07-06')

100%|██████████| 4238/4238 [00:18<00:00, 233.03it/s]


In [3]:
df = KS11_2021_2023.dropna()   # 결측치 제거

# 새로운 칼럼 생성
# (Price : 당일 대비 다음날 주가가 상승했으면 1, 하락했으면 0 표시)
df['Price'] = 0
for i in range(0, 120):
    if df['Close'][i] < df['Close'][i+1]:
        df['Price'][i] = 1
    else:
        df['Price'][i] = 0

# 파일 저장
df.to_csv('./kospi_주가데이터.csv')
df

/var/folders/j6/q03gf6y53_72jb_wzwbp2b140000gn/T/ipykernel_23224/3269824745.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 0
/var/folders/j6/q03gf6y53_72jb_wzwbp2b140000gn/T/ipykernel_23224/3269824745.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = 1
/var/folders/j6/q03gf6y53_72jb_wzwbp2b140000gn/T/ipykernel_23224/3269824745.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] 

,Open,High,Low,Close,Adj Close,Volume,Price
Date,,,,,,,
2021-06-28,3305.459961,3311.689941,3291.300049,3301.889893,3301.889893,1427400,0
2021-06-29,3305.260010,3305.419922,3276.629883,3286.679932,3286.679932,1211800,1
2021-06-30,3288.629883,3305.729980,3288.629883,3296.679932,3296.679932,1608900,0
2021-07-01,3297.919922,3300.120117,3278.100098,3282.060059,3282.060059,1157700,0
2021-07-02,3282.810059,3295.679932,3278.189941,3281.780029,3281.780029,1019400,1
...,...,...,...,...,...,...,...
2023-06-29,2564.020020,2577.689941,2550.020020,2550.020020,2550.020020,602500,0
2023-06-30,2558.429932,2571.030029,2541.719971,2564.280029,2564.280029,508800,0
2023-07-03,2580.889893,2604.629883,2580.489990,2602.469971,2602.469971,615500,0


In [4]:
df.reset_index(names='day', inplace=True)

In [5]:
df = df.astype({'day':'string'})
news_data = news_data.astype({'date':'string'})

In [6]:
news_data['date'][0][0:10]

'2021-09-30'

In [7]:
df['day'][0][0:10]

'2021-06-28'

In [8]:
news_data['day'] = 0

In [9]:
for i in range(len(news_data)):
    news_data['day'][i] = news_data['date'][i][0:10]

/var/folders/j6/q03gf6y53_72jb_wzwbp2b140000gn/T/ipykernel_23224/2433535406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['day'][i] = news_data['date'][i][0:10]


TypeError: 'NAType' object is not subscriptable

In [ ]:
df = df[['day', 'Close', 'Price']]

In [ ]:
df

,day,Close,Price
0,2021-06-28,3301.889893,0
1,2021-06-29,3286.679932,1
2,2021-06-30,3296.679932,0
3,2021-07-01,3282.060059,0
4,2021-07-02,3281.780029,1
...,...,...,...
493,2023-06-29,2550.020020,0
494,2023-06-30,2564.280029,0
495,2023-07-03,2602.469971,0
496,2023-07-04,2593.310059,0


In [ ]:
news_data = pd.merge(news_data, df, how='inner', on='day')

In [ ]:
news_data.sort_values(by='day', ascending=True)

,date,title,link,content,day,Close,Price
1152,2021-09-09 16:30:40,"""울산공항 폐항 검토"" 울산시장 공개 발언에 지역사회 '술렁'",https://n.news.naver.com/mnews/article/001/001...,민간 주도 운영과 지역 사회에 기반한 사회적가치 창출 등으로 해외에서는 사회적기업 ...,2021-09-09,3114.699951,1
1151,2021-09-09 14:37:05,"대한주택건설협회, 사회복지시설에 5650만원 지원",https://n.news.naver.com/mnews/article/031/000...,제회 부산국제영화제 개막식 사회자로 배우 송중기와 박소담이 확정됐다배우 송중기와 ...,2021-09-09,3114.699951,1
1138,2021-09-09 16:40:28,한국에너지공대 세계 10 공대 육성 '지역사회 한 뜻',https://n.news.naver.com/mnews/article/003/001...,美 한국일본 방위에 대한 약속은 여전히 확고 북한 국방과학원은 지난 일일 새로...,2021-09-09,3114.699951,1
1139,2021-09-09 18:47:03,"""언중법 여야 협의체에 기대없다…사회적 합의기구 필요""",https://n.news.naver.com/mnews/article/018/000...,한국협회가 의 사회적 가치를 높이고 활동 및 소통에 대한 인식 향상을 위해 지속...,2021-09-09,3114.699951,1
1141,2021-09-09 16:42:16,"주건협, 추석 앞두고 사회복지시설에 후원금·물품 전달",https://n.news.naver.com/mnews/article/421/000...,칼텍스 년째 여수지역 소외이웃에 한가위 온정 나눔 한화솔루션 생활용품 꾸러미 만들어...,2021-09-09,3114.699951,1
...,...,...,...,...,...,...,...
91,2021-09-30 10:58:03,"바이든 인프라·사회복지 예산안, 의회 통과 앞두고 진통",https://n.news.naver.com/mnews/article/018/000...,서울시 어르신 복지 종합계획 발표구직 지원 어르신일자리지원센터 설립오세훈 시장 어르...,2021-09-30,3068.820068,0
90,2021-09-30 12:24:02,"한국전쟁고아 부부 143억 美 일리노이대 기부 ""큰 돈이 재앙 부를 것 같아 사회 환원""",https://n.news.naver.com/mnews/article/009/000...,제회 부산국제영화제가 다음달 일 오후 시 영화의전당 야외극장에서 진행되는 폐막식 사...,2021-09-30,3068.820068,0
89,2021-09-30 19:06:53,"""미래 수소사회 청사진을 엿보세요""…창원시, '수소빌리티 로드쇼 전시장' 개장",https://n.news.naver.com/mnews/article/277/000...,서울연합뉴스 법무부가 아프간 특별기여자들을 대상으로 한 사회정착교육프로그램을...,2021-09-30,3068.820068,0
95,2021-09-30 08:24:26,[뉴스체크｜사회] 평택 아파트 화재…주민 연기 흡입,https://n.news.naver.com/mnews/article/437/000...,시멘트협회 지역사회 상생 발전 사회공헌 확대억 원 규모 사회공헌 기금 조성생산성본부...,2021-09-30,3068.820068,0


In [ ]:
news_data[news_data['Price']==1].notnull().sum()

date       568
title      568
link       568
content    568
day        568
Close      568
Price      568
dtype: int64

In [ ]:
# 댓글들(comments)에서 하나씩 가져와 reply라는 변수에 list로 저장

replies = []
i = 0
for comment in contents:
  replies.append(text_clean(comment))
  i += 1
  news_data['content'][i-1] = replies[i-1]
print(replies[i-1])

C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2236653096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]


C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2236653096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2236653096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2236653096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][

서울뉴시스 권창회 기자  서울시사회서비스원지부 조합원들이 일 오전 서울 중구 서울시청 앞에서 열린 돌봄서비스 공공성 확보와 돌봄노동자 처우 개선을 위한 서울시 대책 마련 촉구 기자회견에서 손피켓을 들고 있다  사진 영상 제보받습니다 공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다 뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부로 보내주시면 적극 반영하겠습니다


C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2236653096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2236653096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][i-1] = replies[i-1]
C:\Users\USER\AppData\Local\Temp\ipykernel_10648\2236653096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['content'][

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
from konlpy.tag import Okt

# 감성 어휘 사전 로드
sentiment_lexicon = pd.read_csv('./SentiWord_Dict.csv', encoding='utf-8')
sentiment_words = set(sentiment_lexicon['text'])

# 형태소 분석기 초기화
okt = Okt()

# 뉴스 텍스트 감성 분석
sentiments = []
for text in news_data['content']:  # 뉴스 텍스트가 있는 열의 이름에 맞게 수정
    tokens = okt.morphs(text)
    sentiment_score = 0
    for token in tokens:
        if token in sentiment_words:
            sentiment_score += sentiment_lexicon[sentiment_lexicon['text'] == token]['sentiment'].values[0]
    sentiment = '긍정' if sentiment_score > 0 else '부정' if sentiment_score < 0 else '중립'
    sentiments.append(sentiment)

# 감성 결과 출력
for i, sentiment in enumerate(sentiments):
    print(f"뉴스 제목: {news_data['title'][i]}")  # 뉴스 텍스트가 있는 열의 이름에 맞게 수정
    print(f"뉴스 내용: {news_data['content'][i]}")  # 뉴스 텍스트가 있는 열의 이름에 맞게 수정
    print(f"감성 분류: {sentiment}")
    
    print()
    print('-'*50)



JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
news_data['sentiment'] = sentiments

NameError: name 'sentiments' is not defined

In [ ]:
# 데이터 매핑을 위한 딕셔너리 생성
mapp = {'부정': -1, '중립': 0, '긍정': 1}

# '숫자' 열의 데이터를 매핑하여 int 형식으로 변환
news_data['sentiment'] = news_data['sentiment'].map(mapp).astype(int)

In [ ]:
news_data

,date,title,link,content,sentiment
0,2022-06-30 01:13:10,"최저임금 인상에 경제계 ""현실외면한 결정…인상률 동의 어려워""",https://n.news.naver.com/mnews/article/001/001...,경총대한상의중기중앙회소공연 반발부작용 완화 방안 마련해야서울연합뉴스 신선미 김철선 ...,-1
1,2022-06-30 16:07:01,9620원…경제수장도 ‘6% 물가’ 본다는데 5% 오른 최저임금,https://n.news.naver.com/mnews/article/028/000...,올해 시급 원보다 원 인상 물가상승 전망 내년 더 오를 듯 만천명만천명 최저임금 영...,0
2,2022-06-30 20:22:01,"尹 ""한국전 참전에 고마움…경제 안보 협력 강화""",https://n.news.naver.com/mnews/article/079/000...,북대서양조약기구나토 정상회의 참석차 스페인 마드리드를 방문 중인 윤석열 대통령이 일...,1
3,2022-06-30 18:18:47,"尹, 나토 일정 마지막날 '경제 외교'...연기된 나토 총장 면담",https://n.news.naver.com/mnews/article/052/000...,앵커나토정상회의 참석차 스페인을 방문 중인 윤석열 대통령의 첫 해외 일정 나흘째이자...,1
4,2022-06-30 18:11:02,치솟는 가계대출 금리… 한 총리 “경제전쟁 상황”,https://n.news.naver.com/mnews/article/022/000...,월 금리 년개월래 최고저신용 대출 비중 늘어 신용금리 급등 탓 정부 가계이자 상환...,1
...,...,...,...,...,...
2193,2022-06-16 12:20:37,경제계 한국전 참전국 초청 감사행사 기념촬영,https://n.news.naver.com/mnews/article/003/001...,서울뉴시스 배훈식 기자 허창수왼쪽 두번째부터 전국경제인연합회 회장과 이종섭 국방부...,1
2194,2022-06-24 11:36:06,"경제보다 외교에 승부수 … 기시다, 참의원선거 전략",https://n.news.naver.com/mnews/article/021/000...,글로벌 피플기시다 후미오岸田文雄사진 일본 총리가 과감한 외교 행보를 보이고 있다기...,1
2195,2022-06-23 22:06:12,"윤 대통령, '한국 경제학계 거목' 조순 전 부총리 빈소 조문",https://n.news.naver.com/mnews/article/421/000...,서울뉴스 안은나 기자 윤석열 대통령이 일 송파구 서울아산병원에 마련된 조순 전 경...,0
2196,2022-06-16 14:07:02,[尹정부 경제정책] 법인세 최고세율 22%로 인하…1주택자 14억까지 종부세 비과세,https://n.news.naver.com/mnews/article/016/000...,에 몰린 형벌규정 행정제재로 전환정년 연장 논의 가시화증권거래세 로 인하연금저축 연...,-1
